In [ ]:
%cd /content/drive/MyDrive/Assignment_2/

/content/drive/MyDrive/Assignment_2


In [ ]:
!pip install libsvm

In [ ]:
from libsvm.svmutil import *

In [ ]:
import numpy as np
import os
import csv
import copy
import random
import math
import matplotlib.pyplot as plt

In [ ]:
#convert data to SVM format
train_x=[]
train_y=[]
test_x=[]
test_y=[]
n=0

train_idx=random.sample(range(700),300)
with open('health_data.csv','r') as file:
  csvf= csv.DictReader(file)
  data=[]
  for row in csvf:
    data=list(row.values())
    
    if n in train_idx:

      train_x.append([int(i)/100 for i in data[:-1]])
      train_y.append(1 if int(data[-1])==1 else -1)
    else:
      test_x.append([int(i)/100 for i in data[:-1]])
      test_y.append(1 if int(data[-1])==1 else -1)
    
    n+=1
test_x=np.array(test_x)
train_x=np.array(train_x)
test_y=np.reshape(np.array(test_y),(len(test_y),1))
train_y=np.reshape(np.array(train_y),(len(train_y),1))


In [ ]:
#kernel function allows poly kerne, and rbf kernel
def kernel_f(a,b,degree=None,k_type=None,gamma=None):
  if k_type==None:
    return np.dot(a,b)
  elif k_type=="poly":
    try:
      return (1+np.dot(a,b))**degree
    except:
      print("enter a valid degree please.")
  elif k_type=="RBF":
    # try:
    return math.exp(-np.linalg.norm(a-b)*gamma)
    # except:
    #   print("enter a valid gamma")
  else:
    raise ValueError("Please enter a valid kernel type")


In [ ]:
#evaluates w.x+b using the kernel trick
def f(X,y,xj,alpha,b,k_type,degree,gamma):
  y_hat=0.0
  for i in range(X.shape[0]):
    y_hat+=(alpha[i]*y[i]*(kernel_f(xj,X[i,:],degree,k_type,gamma)))
  return y_hat+b

In [ ]:
#SMO algorithm
def SMO(X,y,C,tol,max_iter,k_type=None,degree=None,gamma=None):
  
  m=y.shape[0]
  
  alpha=np.zeros((m,1))
  b=0
  
  E=np.zeros((m,1))
  a=copy.deepcopy(alpha)
  for iter in range(max_iter):

    numca=0
    
    for i in range(m):
      E[i]=f(X,y,X[i,:],alpha,b,k_type,degree,gamma)-y[i]
      if (y[i][0]*E[i]<-tol and alpha[i]<C) or (y[i][0]*E[i]>tol and alpha[i]>0) :
        j=random.randrange(m)
        while(j==i):
          j=random.randrange(m)
        E[j]=f(X,y,X[j,:],alpha,b,k_type,degree,gamma)-y[j]

        a[i]=alpha[i]
        a[j]=alpha[j]
        if y[i]!=y[j]:
          L=max(0,alpha[j]-alpha[i])
          H=min(C,C+alpha[j]-alpha[i])
        else:
          L=max(0,alpha[j]+alpha[i]-C)
          H=min(C,alpha[j]+alpha[i])
        if L==H:
          continue
        n=2*kernel_f(X[i, :], X[j, :],degree,k_type,gamma) - kernel_f(X[i, :], X[i, :],degree,k_type,gamma) - kernel_f(X[j, :], X[j, :],degree,k_type,gamma)
        if n>=0:
          continue
        alpha[j]=alpha[j]-((y[j][0]*(E[i]-E[j]))/n)
        if alpha[j] > H:
          alpha[j] = H
        elif alpha[j] < L:
          alpha[j] = L
        if abs(alpha[j] - a[j]) < tol:
          continue
        alpha[i] = alpha[i] + y[i]*y[j]*(a[j] - alpha[j])
        b1 = b - E[i] - (y[i] * (alpha[i] - a[i]) * kernel_f(X[i, :], X[i, :],degree,k_type,gamma)) - \
          (y[j] * (alpha[j] - a[j]) * kernel_f(X[i, :], X[j, :],degree,k_type,gamma))
        b2 = b - E[j] - (y[i] * (alpha[i] - a[i]) * kernel_f(X[i, :], X[j, :],degree,k_type,gamma)) - \
          (y[j] * kernel_f(X[j, :], X[j, :],degree,k_type,gamma) * (alpha[j] - a[j]))
        if a[i] > 0 and a[i] < C:
          b = b1
        elif a[j] > 0 and a[j] < C:
          b = b2
        else:
          b = (b1 + b2) / 2
        numca += 1
        if numca == 0:
          passes += 1
        else:
          passes = 0
  return alpha,b

In [ ]:
# predicts the class of a particular sample
def predict(x,kernel=None,degree=None,gamma=None):
  a,b=SMO(train_x,train_y,1,1e-5,10,kernel,degree,gamma)
  y_hat=f(train_x,train_y,x,a,b,kernel,degree,gamma)
  return np.sign(y_hat)

In [ ]:
#predicts class of whole test_dataset
def res(k_type=None,degree=None,gamma=None):
  a,b=SMO(train_x,train_y,1,1e-5,10,k_type,degree,gamma)
  y=np.zeros(test_x.shape[0])
  for i in range(test_x.shape[0]):
    y[i]=np.sign(f(train_x,train_y,test_x[i,:],a,b,k_type,degree,gamma))
  return y,a,b

In [ ]:
#finding time for our SMO algo
smp=[]
from timeit import default_timer as timer
for i in range(20):
  start = timer()
  SMO(train_x[:700*i//20,:],train_y[:700*i//20],1,1e-5,10,None,None,None)
  end = timer()
  smp.append(end - start)

In [ ]:
#finding time for LibSBM SMO algo
libs=[]
params=svm_parameter('-t 0')
from timeit import default_timer as timer
for i in range(1,20):
  start = timer()
  prob=svm_problem((train_y[:700*i//20]).reshape(700*i//20),train_x[:700*i//20,:],isKernel=False)
  m=svm_train(prob,params)
  end = timer()
  libs.append(end - start)

In [ ]:
#plotting the graphs
x=[700*i//20 for i in range(20)]
import matplotlib.pyplot as plt
plt.plot(x,smp)
# plt.plot(x[1:],libs)


In [ ]:
plt.plot(x[1:],libs)